In [2]:
import cv2
import mediapipe as mp
import numpy as np

# mp setup
mp_draw = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# quick angle calc
def knee_angle(p1, p2, p3):
    if p1.visibility < 0.8 or p2.visibility < 0.8 or p3.visibility < 0.8:
        return -1
    a = np.array([p1.x - p2.x, p1.y - p2.y, p1.z - p2.z])
    b = np.array([p3.x - p2.x, p3.y - p2.y, p3.z - p2.z])
    ang = np.degrees(np.arccos(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))))
    return 360 - ang if ang > 180 else ang

# leg pos: 1 squat, 2 mid, 3 up
def leg_pos(angle):
    if angle < 0: return 0
    if angle < 105: return 1
    if angle < 150: return 2
    return 3

# cam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("no cam!")
    exit()

# vars
count = 0
last_state = 9

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ok, frame = cap.read()
        if not ok:
            print("cam error")
            break

        frame = cv2.resize(frame, (960, 540))
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb.flags.writeable = False
        results = pose.process(rgb)
        rgb.flags.writeable = True

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark

            mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            r_ang = knee_angle(lm[24], lm[26], lm[28])
            l_ang = knee_angle(lm[23], lm[25], lm[27])

            rs = leg_pos(r_ang)
            ls = leg_pos(l_ang)
            state = rs * ls

            if state in (1, 9) and state != last_state:
                last_state = state
                if state == 1:
                    count += 1
                    print(f"squat! total: {count}")
            elif state == 0:
                if rs == 0: print("right leg lost")
                if ls == 0: print("left leg lost")

        else:
            print("step in view")

        cv2.putText(frame, f"Reps: {count}", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Squat Counter", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost
left leg lost
right leg lost

In [3]:
import cv2
import mediapipe as mp
import numpy as np

mp_draw = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def knee_angle(p1, p2, p3):
    if p1.visibility < 0.8 or p2.visibility < 0.8 or p3.visibility < 0.8:
        return -1
    a = np.array([p1.x - p2.x, p1.y - p2.y, p1.z - p2.z])
    b = np.array([p3.x - p2.x, p3.y - p2.y, p3.z - p2.z])
    ang = np.degrees(np.arccos(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))))
    return 360 - ang if ang > 180 else ang

def leg_pos(angle):
    if angle < 0: return 0
    if angle < 105: return 1
    if angle < 150: return 2
    return 3

# Map angle to depth percent
def squat_depth_percent(angle, max_angle=170, min_angle=90):
    if angle < 0:
        return 0
    # Clamp angle range
    angle = max(min_angle, min(max_angle, angle))
    # Invert so smaller angle -> bigger percent
    return int(((max_angle - angle) / (max_angle - min_angle)) * 100)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No camera!")
    exit()

count = 0
last_state = 9

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ok, frame = cap.read()
        if not ok:
            print("Camera error")
            break

        frame = cv2.resize(frame, (960, 540))
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb.flags.writeable = False
        results = pose.process(rgb)
        rgb.flags.writeable = True

        depth_percent = 0  # Default depth

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            r_ang = knee_angle(lm[24], lm[26], lm[28])
            l_ang = knee_angle(lm[23], lm[25], lm[27])

            rs = leg_pos(r_ang)
            ls = leg_pos(l_ang)
            state = rs * ls

            # Use average of both legs for depth
            if r_ang > 0 and l_ang > 0:
                avg_angle = (r_ang + l_ang) / 2
                depth_percent = squat_depth_percent(avg_angle)

            if state in (1, 9) and state != last_state:
                last_state = state
                if state == 1:
                    count += 1
                    print(f"Squat! Total: {count}")
            elif state == 0:
                if rs == 0: print("Right leg lost")
                if ls == 0: print("Left leg lost")

        else:
            print("Step into view")

        # Draw squat counter
        cv2.putText(frame, f"Reps: {count}", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw depth progress bar
        bar_x = frame.shape[1] - 50
        bar_height = int((depth_percent / 100) * frame.shape[0])
        cv2.rectangle(frame, (bar_x, frame.shape[0] - bar_height),
                      (bar_x + 30, frame.shape[0]), (0, 255, 0), -1)
        cv2.rectangle(frame, (bar_x, 0), (bar_x + 30, frame.shape[0]), (255, 255, 255), 2)
        cv2.putText(frame, f"{depth_percent}%", (bar_x - 10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        cv2.imshow("Squat Counter", frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost
Left leg lost
Right leg lost